In [104]:
import requests
import pandas as pd
import json

In [105]:
API_KEY = "062fcaf65c7a4f1e56230ca193f72e9b-70d22f09f9d48c22fe72aabf5b1821e9"
ACCOUND_ID = "101-012-21494048-001"
OANDA_URL = "https://api-fxpractice.oanda.com/v3"

In [106]:
session = requests.Session()

In [107]:
# autorizziamo la richiesta
session.headers.update({
    'Authorization': 'Bearer ' + API_KEY,
    'Content-Type': 'application/json'
    })

In [108]:
params = dict(
    count = 10,
    granularity = "M1",
    price= "MBA"
)

In [109]:
url = f'{OANDA_URL}/accounts/{ACCOUND_ID}/instruments'

In [110]:
res = session.get(url, params=params, data= None, headers=None)

In [111]:
res.status_code

200

In [112]:
data = res.json()

In [113]:
instrument_list = data['instruments']


In [114]:
key_i = ['name', 'type', 'displayName', 'pipLocation', 'displayPrecision', 'tradeUnitsPrecision', 'marginRate' ]

In [115]:
url = f'{OANDA_URL}/accounts/{ACCOUND_ID}/instruments?instruments=BTC_USD'
res = session.get(url, params=params, data=None, headers=None)


In [116]:
data = res.json()


In [117]:
instrument_list.append(data['instruments'][0])

In [118]:
instrument_dict = {}
for i in instrument_list:
    key = i['name']
    instrument_dict[key] = { k: i[k] for k in key_i}



In [119]:
with open('../data/instrument.json', 'w') as f:
    f.write(json.dumps(instrument_dict, indent=2))